In [3]:
import pandas as pd
import json
import requests 
import time
import random
import tqdm
import os

In [2]:
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Accept':'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8,th;q=0.7,zh-CN;q=0.6,zh;q=0.5',
    'X-Guest-Token': 'Ae1PtIuUDbozkgpQ6VOW57jKqvchZxF2',
    'Connection' : 'keep-alive',
    'TE': 'Trailers'
}

params = {
    'limit':'5',
    'include': 'comments,contribute_info,attribute_vote_summary',
}

In [27]:
def product_parser(json):
    pInfo = {}

    pInfo['id'] = json.get('id')
    pInfo['sku'] = json.get('sku')
    pInfo['name'] = json.get('name')
    pInfo['discription'] = json.get('short_description')
    pInfo['original_price'] = json.get('original_price')
    pInfo['list_price'] = json.get('list_price')
    pInfo['price'] = json.get('price')
    pInfo['alltime_quantity_sold'] = json.get('all_time_quantity_sold',{})
    pInfo['quantity_sold'] = json.get('quantity_sold', {}).get('value')
    pInfo['inventory_status'] = json.get('inventory_status')
    pInfo['fulfillment_type'] = json.get('inventory', {}).get('fulfillment_type')
    pInfo['brand'] = json.get('brand', {}).get('name')
    pInfo['category_id'] = json.get('categories', {}).get('id')
    pInfo['category'] = json.get('categories').get('name')
    pInfo['review_count'] = json.get('review_count')
    pInfo['rating_average'] = json.get('rating_average')
    pInfo['favourite_count'] = json.get('favourite_count')
    pInfo['ASA_cashback'] = json.get('asa_cashback_widget', {}).get('text')
    pInfo['pay_later'] = json.get('installment_info_v2',{})
    pInfo['current_seller'] = json.get('current_seller', {}).get('name')
    pInfo['date_created'] = json.get('day_ago_created')
    pInfo['video_url'] = json.get('video_url',{})

    return pInfo


In [4]:
folder = 'D:\TIKI\GIAY-NAM(done)'
category_list = []
for file_name in os.listdir(folder):
    file_path = os.path.join(folder,file_name)
    id_list = pd.read_csv(file_path,usecols=['product_id','category_name'])
    category_list.append(id_list)

category_list_df = pd.concat(category_list)
category_list_df.to_csv('giay-nam-id.csv')

In [5]:
category_list_df = category_list_df.drop_duplicates()

In [6]:
category_list_final = category_list_df['product_id'].to_list()

In [7]:
response = requests.get('https://tiki.vn/api/v2/products/164789853',headers=headers,params=params)

In [8]:
category_product_info = []
error_list = []

for id in (category_list_final):
    try:
        response = requests.get('https://tiki.vn/api/v2/products/{}'.format(id), headers=headers, params=params)
        if response.status_code == 200:
            print('crawling product id {}'.format(id))
            category_product_info.append(product_parser(response.json()))
        else:
            print('cannot crawl product id {}!'.format(id))
            error_list.append(id)
    except Exception as e:
        print('An error occurred while crawling product id {}: {}'.format(id, e))
        error_list.append(id)
        continue


crawling product id 103682599
crawling product id 176818958
crawling product id 119214657
crawling product id 164789853
crawling product id 208440729
crawling product id 254207022
crawling product id 259538898
crawling product id 146045291
crawling product id 98297802
crawling product id 95486564
crawling product id 192796419
crawling product id 181237205
crawling product id 124778290
crawling product id 143363213
crawling product id 162782918
crawling product id 68490286
crawling product id 176818787
crawling product id 86379293
crawling product id 170443283
crawling product id 144181471
crawling product id 125781054
crawling product id 117320164
crawling product id 86328492
crawling product id 153258290
crawling product id 170442567
crawling product id 151751324
crawling product id 202863335
crawling product id 242703595
crawling product id 147777760
crawling product id 244111699
crawling product id 242704307
crawling product id 171127233
crawling product id 140967559
crawling produc

In [13]:
giay_nam_df = pd.DataFrame(category_product_info)
giay_nam_df.to_csv('giay-nam.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11086 entries, 0 to 11085
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     11086 non-null  int64  
 1   sku                    11086 non-null  object 
 2   name                   11086 non-null  object 
 3   discription            11086 non-null  object 
 4   original_price         11086 non-null  int64  
 5   list_price             11086 non-null  int64  
 6   price                  11086 non-null  int64  
 7   alltime_quantity_sold  11086 non-null  object 
 8   quantity_sold          3895 non-null   float64
 9   inventory_status       11086 non-null  object 
 10  fulfillment_type       11086 non-null  object 
 11  brand                  11086 non-null  object 
 12  category_id            11086 non-null  int64  
 13  category               11086 non-null  object 
 14  review_count           11086 non-null  int64  
 15  ra

In [28]:
while error_list:
    error_list_remained = []
    
    for id in error_list:
        try:
            response = requests.get('https://tiki.vn/api/v2/products/{}'.format(id), headers=headers, params=params)
            if response.status_code == 200:
                print('crawling product id {}'.format(id))
                category_product_info.append(product_parser(response.json()))
            else:
                print('cannot crawl product id {}!'.format(id))
                error_list_remained.append(id)
        except Exception as e:
            print('An error occurred while crawling product id {}: {}'.format(id, e))
            error_list_remained.append(id)
            continue
    if len(error_list_remained) == len(error_list):
        break      
    else:  
        error_list = error_list_remained 
        time.sleep(1)


crawling product id 208497043
An error occurred while crawling product id 208497043: 'NoneType' object has no attribute 'get'
crawling product id 208493977
An error occurred while crawling product id 208493977: 'NoneType' object has no attribute 'get'
crawling product id 174424850
An error occurred while crawling product id 174424850: 'NoneType' object has no attribute 'get'
crawling product id 76164811
An error occurred while crawling product id 76164811: 'NoneType' object has no attribute 'get'
crawling product id 263196067
An error occurred while crawling product id 263196067: 'NoneType' object has no attribute 'get'
crawling product id 127580978
An error occurred while crawling product id 127580978: 'NoneType' object has no attribute 'get'
crawling product id 180318533
An error occurred while crawling product id 180318533: 'NoneType' object has no attribute 'get'
crawling product id 222663683
An error occurred while crawling product id 222663683: 'NoneType' object has no attribute 

In [29]:
print(error_list)

[208497043, 208493977, 174424850, 76164811, 263196067, 127580978, 180318533, 222663683, 199198441]
